In [28]:
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv("train.csv")
data.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [29]:
data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   date         object 
 2   store_nbr    int64  
 3   family       object 
 4   sales        float64
 5   onpromotion  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 137.4+ MB


,id,store_nbr,sales,onpromotion
count,3.000888e+06,3.000888e+06,3.000888e+06,3.000888e+06
mean,1.500444e+06,2.750000e+01,3.577757e+02,2.602770e+00
std,8.662819e+05,1.558579e+01,1.101998e+03,1.221888e+01
min,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
25%,7.502218e+05,1.400000e+01,0.000000e+00,0.000000e+00
50%,1.500444e+06,2.750000e+01,1.100000e+01,0.000000e+00
75%,2.250665e+06,4.100000e+01,1.958473e+02,0.000000e+00
max,3.000887e+06,5.400000e+01,1.247170e+05,7.410000e+02


In [30]:
holidays = pd.read_csv("holidays_events.csv")
holidays.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [31]:
oil = pd.read_csv("oil.csv")
oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [91]:
data1 = data.merge(holidays, how="left",left_on="date",right_on="date")
data1= data1.merge(oil, how="left",left_on="date",right_on="date")


In [92]:
data1["date"]= pd.to_datetime(data1["date"])
data1["month"] = data1.apply(lambda x:x["date"].month,axis=1)
data1["year"] = data1.apply(lambda x:x["date"].year,axis=1)
data1["day"] = data1.apply(lambda x:x["date"].day,axis=1)

In [94]:
data1.fillna(0,inplace=True)
data1.head()


,id,date,store_nbr,family,sales,onpromotion,type,locale,locale_name,description,transferred,dcoilwtico,month,year,day
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Holiday,National,Ecuador,Primer dia del ano,False,0.0,1,2013,1
1,1,2013-01-01,1,BABY CARE,0.0,0,Holiday,National,Ecuador,Primer dia del ano,False,0.0,1,2013,1
2,2,2013-01-01,1,BEAUTY,0.0,0,Holiday,National,Ecuador,Primer dia del ano,False,0.0,1,2013,1
3,3,2013-01-01,1,BEVERAGES,0.0,0,Holiday,National,Ecuador,Primer dia del ano,False,0.0,1,2013,1
4,4,2013-01-01,1,BOOKS,0.0,0,Holiday,National,Ecuador,Primer dia del ano,False,0.0,1,2013,1


In [109]:
data2 = pd.get_dummies(data1,columns=["family","type","transferred"])

data2.drop(["date","id","locale","locale_name","description","type_Additional","type_Bridge","type_Event","type_Transfer","transferred_True","type_0",
"type_Work Day","transferred_False"],axis=1,inplace=True)
data2.head()

,store_nbr,sales,onpromotion,dcoilwtico,month,year,day,family_AUTOMOTIVE,family_BABY CARE,family_BEAUTY,...,family_MEATS,family_PERSONAL CARE,family_PET SUPPLIES,family_PLAYERS AND ELECTRONICS,family_POULTRY,family_PREPARED FOODS,family_PRODUCE,family_SCHOOL AND OFFICE SUPPLIES,family_SEAFOOD,type_Holiday
0,1,0.0,0,0.0,1,2013,1,True,False,False,...,False,False,False,False,False,False,False,False,False,True
1,1,0.0,0,0.0,1,2013,1,False,True,False,...,False,False,False,False,False,False,False,False,False,True
2,1,0.0,0,0.0,1,2013,1,False,False,True,...,False,False,False,False,False,False,False,False,False,True
3,1,0.0,0,0.0,1,2013,1,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,1,0.0,0,0.0,1,2013,1,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [110]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split 

X = data2.drop(["sales"],axis=1)
y = data2["sales"]

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.20, random_state=42)

model = LinearRegression()
model.fit(X_train,y_train)

/Users/shreyarora/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/shreyarora/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/shreyarora/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/shreyarora/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version

LinearRegression()

In [111]:
mean_squared_error(y_test,model.predict(X_test))

/Users/shreyarora/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/shreyarora/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/shreyarora/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/shreyarora/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version

545054.8454004147

In [112]:
test_data = pd.read_csv("test.csv")
test_data1 = test_data.merge(holidays, how="left",left_on="date",right_on="date")
test_data1= test_data1.merge(oil, how="left",left_on="date",right_on="date")

In [113]:
test_data1["date"]= pd.to_datetime(test_data1["date"])
test_data1["month"] = test_data1.apply(lambda x:x["date"].month,axis=1)
test_data1["year"] = test_data1.apply(lambda x:x["date"].year,axis=1)
test_data1["day"] = test_data1.apply(lambda x:x["date"].day,axis=1)

In [123]:
test_data1.fillna(0,inplace=True)
test_data2 = pd.get_dummies(test_data1,columns=["family","type","transferred"])
test_data2.drop(["date","id","locale","locale_name","description","transferred_0","type_0"],axis=1,inplace=True)
X_new = test_data2
predictions = model.predict(X_new)
test_data["sales"] = predictions

test_data[["id","sales"]].to_csv("submission.csv",index=False)

/Users/shreyarora/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/shreyarora/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/shreyarora/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
